<a href="https://colab.research.google.com/github/Taibur-Raxon/Python/blob/main/07a_Grouping_and_merging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import the Pandas library and create DataFrame

Before doing anything else, you'll need to import Pandas and get some data to work with.

In [2]:
import pandas as pd

In [3]:
data = {
    'Name': ['Alice', 'Bob', 'Claire', 'David', 'Emma', 'Alice', 'David'],
    'Age': [25, 30, 22, 28, 35, 25, 28],
    'Department': ['IT', 'HR', 'Finance', 'IT', 'Marketing', 'HR', 'IT'],
    'Gender': ['Female', 'Male', 'Female', 'Male', 'Female', 'Female', 'Male'],
    'Salary': [50000, 60000, 45000, 70000, 80000, 52000, 72000],
    'Rating': [4.5, 3.8, 4.2, 4.0, 4.7, 4.3, 3.9]
}

df = pd.DataFrame(data)
df

,Name,Age,Department,Gender,Salary,Rating
0,Alice,25,IT,Female,50000,4.5
1,Bob,30,HR,Male,60000,3.8
2,Claire,22,Finance,Female,45000,4.2
3,David,28,IT,Male,70000,4.0
4,Emma,35,Marketing,Female,80000,4.7
5,Alice,25,HR,Female,52000,4.3
6,David,28,IT,Male,72000,3.9


## 1. Grouping

As a Data Scientist, you'll often wish to know less about how every entity looks than how they look in aggregates.
- You can't know *each customer's* favorite time to shop, but you can know when *customers in different age groups* shop most often.
- You may not wish to know how *every product* sells, but rather how *products in different departments* are selling right now.

Grouping data by some feature allows you to collect those entries sharing some common feature and evaluate them collectively.

### 1.1 `.groupby()`

The first step in grouping with Pandas is to choose which feature (i.e. which column) will be be used to determine the groups.

In [ ]:
df.groupby("Department")

**What is this strange output?**

By itself, `.groupby()` will only lay the groundwork for grouping. To get any useful information, you'll need to specifiy some sort of aggregate: A way to describe many entries at once.

### 1.2 Aggregate functions
From your work with statistics, you will already know several aggregate functions, such as mean, median or standard deviation.  

Pandas provides several built-in functions to summarize data, some of which you'll find listed below. While many methods can be applied to Series or DataFrame as a whole, today we'll how they can be used to describe grouped data.

| Aggregate function | Description                                    |
|---------------------|------------------------------------------------|
| [count()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.count.html)             | Returns count for each group                   |
| [size()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.size.html?highlight=size#pandas.DataFrame.size)             | Returns size for each group                    |
| [sum()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sum.html)               | Returns total sum for each group               |
| [mean() ](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.mean.html)              | Returns mean for each group. Same as average() |
| [std()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.std.html)               | Returns standard deviation for each group      |
| [var() ](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.var.html)              | Returns variance for each group                      |
| [describe()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.describe.html)          | Returns different descriptive statistics                   |
| [min() ](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.min.html)               | Returns minimum value for each group           |
| [max() ](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.max.html#pandas.DataFrame.max)              | Returns maximum value for each group           |

Applied to a DataFrame, `.mean()` will return the mean of each numeric column.

In [ ]:
df.mean(numeric_only=True)

Age          27.571429
Salary    61285.714286
Rating        4.200000
dtype: float64

The same method can be used on a single Series to get its mean.

In [ ]:
df['Age'].mean()

27.571428571428573

The method `.count()` will simply count the number of non-null entries.

In [ ]:
df['Age'].count()

7

Now let's see what happens if we use `.mean()` after grouping:

In [ ]:
df.groupby('Department').mean(numeric_only=True)

,Age,Salary,Rating
Department,,,
Finance,22.0,45000.0,4.200000
HR,27.5,56000.0,4.050000
IT,27.0,64000.0,4.133333
Marketing,35.0,80000.0,4.700000


Let's break down what we see here:
- The index has changed. Now each **unique** value from the "Department" column has become an index.
- Each column now represents the mean of that feature *with respect to the __group__*.
- Since only one person works in Finance, we see the same numbers as appear on line 2 of `df`.
- HR has two members — earning 52,000 and 60,000 per year — so the grouped DataFrame shows a mean earning of 56,000 in HR.

#### 1.2.1 Aggregating on one column

To see aggregate data about a single column, you can select that column with the indexing operator after performing the grouping.

In [ ]:
df.groupby('Gender')['Age'].mean()

Gender
Female    26.750000
Male      28.666667
Name: Age, dtype: float64

#### 1.2.2 Aggregating on two or more columns
Multiple columns can be chosen to perform one aggregate function on by giving a list of column labels to the indexing operator

In [ ]:
df.groupby('Gender')[['Age','Salary']].mean()

,Age,Salary
Gender,,
Female,26.750000,56750.000000
Male,28.666667,67333.333333


#### 1.2.3 Multiple aggregate functions with `.agg()`
A wonderfully flexible method for performing multiple aggregating functions, .agg() can take a variety of input-formats to fit your needs.

To calculate multiple aggregates on one column, use a list of aggregate functions as strings.

In [ ]:
df.groupby('Department')['Salary'].agg(['mean', 'median'])

,mean,median
Department,,
Finance,45000.0,45000.0
HR,56000.0,56000.0
IT,64000.0,70000.0
Marketing,80000.0,80000.0


Aggregates can be applied to specific columns by using a dictionary of columns as keys and aggregate functions as values.

In [ ]:
df.groupby('Department').agg( {'Age':'max', 'Salary': 'mean'})

,Age,Salary
Department,,
Finance,22,45000.0
HR,30,56000.0
IT,28,64000.0
Marketing,35,80000.0


You can also give a list of functions as a value in your dictionary to perform more than one aggregate function on the same column.

In [ ]:
df.groupby('Department').agg( {'Age':'max', 'Salary': ['mean','max']})

Age   Salary       
           max     mean    max
Department                    
Finance     22  45000.0  45000
HR          30  56000.0  60000
IT          28  64000.0  72000
Marketing   35  80000.0  80000

Output columns can even be given custom names by providing named tuples as arguments.

In [ ]:
df.groupby('Department').agg( age_max=('Age', 'max'), salary_mean=('Salary', 'mean'))

,age_max,salary_mean
Department,,
Finance,22,45000.0
HR,30,56000.0
IT,28,64000.0
Marketing,35,80000.0


### 1.3 Grouping on multiple features

`.groupby()` accepts several columns as a list, which creates groups with subgroups.  

Let's find the average age and salary of each gender inside each department:

In [ ]:
df.groupby(['Department','Gender'])[['Age','Salary']].mean()

Age   Salary
Department Gender               
Finance    Female  22.0  45000.0
HR         Female  25.0  52000.0
           Male    30.0  60000.0
IT         Female  25.0  50000.0
           Male    28.0  71000.0
Marketing  Female  35.0  80000.0

## Merging

In [4]:
new_data = {
    'Name': ['Alice', 'Bob', 'Claire', 'David', 'Emma', 'Alice', 'David', 'Erick'],
    'Age': [25, 30, 22, 28, 35, 25, 28, 21],
    'Department': ['IT', 'HR', 'Finance', 'IT', 'Marketing', 'HR', 'IT', 'Temp'],
    'Gender': ['Female', 'Male', 'Female', 'Male', 'Female', 'Female', 'Male', 'Male'],
    'Salary': [50000, 60000, 45000, 70000, 80000, 52000, 72000, 31000],
    'Rating': [4.5, 3.8, 4.2, 4.0, 4.7, 4.3, 3.9, 4.1]
}
new_df = pd.DataFrame(new_data)

departments_data = {
    'Department': ['HR', 'Finance', 'Engineering', 'Marketing', 'IT'],
    'Location' : ['2nd floor', '2nd floor', '1st floor', '3rd floor', '1st floor'],
    'EstablishedYear': [2000, 1998, 2005, 2010,2002]
}
departments_df = pd.DataFrame(departments_data)
departments_df

,Department,Location,EstablishedYear
0,HR,2nd floor,2000
1,Finance,2nd floor,1998
2,Engineering,1st floor,2005
3,Marketing,3rd floor,2010
4,IT,1st floor,2002


In [ ]:
new_df

,Name,Age,Department,Gender,Salary,Rating
0,Alice,25,IT,Female,50000,4.5
1,Bob,30,HR,Male,60000,3.8
2,Claire,22,Finance,Female,45000,4.2
3,David,28,IT,Male,70000,4.0
4,Emma,35,Marketing,Female,80000,4.7
5,Alice,25,HR,Female,52000,4.3
6,David,28,IT,Male,72000,3.9
7,Erick,21,Temp,Male,31000,4.1


### 2.1 .merge()  


Often data from multiple sources needs to be considered together. Not only seen as one table but actually *aligned*, so that relevant pieces from the two DataFrames end up on the same row.

#### 2.1.1 Syntax

The `.merge()` method takes several agruments, so let's explore them:     
```
df.merge(df_2,
        left_on='df_column',      
        right_on='df_2_column',     
        how='inner')
```  
- `df` is the starting, or "left" DataFrame   
- The first argument, `df_2` is the DataFrame to merge with, also known as the "right"  
- `left_on` represents the column in the left DataFrame to join on  
- `right_on` represents the column in the right DataFrame to join on  
    - These columns should have shared information — values found in both DataFrames.
    - When each DataFrame has an appropriate column with the same name you can also use the single parameter `on` instead of `left_on` and `right_on`  
- `how` represents the type of merge (‘left’, ‘right’, ‘outer’, ‘inner’, ‘cross’). The default type is ‘inner’.

#### 2.1.2 Inner merge  
An inner merge will return only those rows for which matching information is present in *both* DataFrames.


In [ ]:
new_df.merge(departments_df, left_on='Department',right_on='Department',how='inner')

,Name,Age,Department,Gender,Salary,Rating,Location,EstablishedYear
0,Alice,25,IT,Female,50000,4.5,1st floor,2002
1,David,28,IT,Male,70000,4.0,1st floor,2002
2,David,28,IT,Male,72000,3.9,1st floor,2002
3,Bob,30,HR,Male,60000,3.8,2nd floor,2000
4,Alice,25,HR,Female,52000,4.3,2nd floor,2000
5,Claire,22,Finance,Female,45000,4.2,2nd floor,1998
6,Emma,35,Marketing,Female,80000,4.7,3rd floor,2010


Notice that Erick does not appear in the merged DataFrame, because his position, Temp, is not listed in `departments_df`.

Likewise, there is nothing about the Engineering department, because no employee is listed as working there.

#### 2.1.3 Left merge

A left merge will maintain *all* data from the left DataFrame, even if it does not align with a row of the right DataFrame.

In [ ]:
new_df.merge(departments_df, on='Department',how='left')

,Name,Age,Department,Gender,Salary,Rating,Location,EstablishedYear
0,Alice,25,IT,Female,50000,4.5,1st floor,2002.0
1,Bob,30,HR,Male,60000,3.8,2nd floor,2000.0
2,Claire,22,Finance,Female,45000,4.2,2nd floor,1998.0
3,David,28,IT,Male,70000,4.0,1st floor,2002.0
4,Emma,35,Marketing,Female,80000,4.7,3rd floor,2010.0
5,Alice,25,HR,Female,52000,4.3,2nd floor,2000.0
6,David,28,IT,Male,72000,3.9,1st floor,2002.0
7,Erick,21,Temp,Male,31000,4.1,NaN,NaN


This time, Erick made it into the merged DataFrame, and his row shows null values in location and established year.

Again, nothing about Engineering, because nobody seems to work there.

#### 2.1.3 Right merge

A right merge will maintain *all* data from the right DataFrame, even if it does not align with a row of the left DataFrame.

In [12]:
neww_df = new_df.merge(departments_df, on='Department',how='right')
neww_df



,Name,Age,Department,Gender,Salary,Rating,Location,EstablishedYear
0,Bob,30.0,HR,Male,60000.0,3.8,2nd floor,2000
1,Alice,25.0,HR,Female,52000.0,4.3,2nd floor,2000
2,Claire,22.0,Finance,Female,45000.0,4.2,2nd floor,1998
3,NaN,NaN,Engineering,NaN,NaN,NaN,1st floor,2005
4,Emma,35.0,Marketing,Female,80000.0,4.7,3rd floor,2010
5,Alice,25.0,IT,Female,50000.0,4.5,1st floor,2002
6,David,28.0,IT,Male,70000.0,4.0,1st floor,2002
7,David,28.0,IT,Male,72000.0,3.9,1st floor,2002


In [9]:
neww_df.dropna()

,Name,Age,Department,Gender,Salary,Rating,Location,EstablishedYear
0,Bob,30.0,HR,Male,60000.0,3.8,2nd floor,2000
1,Alice,25.0,HR,Female,52000.0,4.3,2nd floor,2000
2,Claire,22.0,Finance,Female,45000.0,4.2,2nd floor,1998
4,Emma,35.0,Marketing,Female,80000.0,4.7,3rd floor,2010
5,Alice,25.0,IT,Female,50000.0,4.5,1st floor,2002
6,David,28.0,IT,Male,70000.0,4.0,1st floor,2002
7,David,28.0,IT,Male,72000.0,3.9,1st floor,2002


This time, a row in the merged DataFrame includes Engineering, but all columns with employee information contain null values.

Erick has been left out again, because there is no department for his Temp position.

## Challenges

In [5]:
data1 = {
    'City': ['New York', 'Los Angeles', 'Chicago', 'Houston', 'San Francisco', 'London', 'Paris', 'Berlin', 'Rome', 'Tokyo'],
    'Country': ['USA', 'USA', 'USA', 'USA', 'USA', 'UK', 'France', 'Germany', 'Italy', 'Japan'],
    'Population (Millions)': [8.4, 3.9, 2.7, 2.3, 0.9, 8.9, 2.1, 3.7, 2.8, 13.9],
    'Area (km2)': [468.9, 502.8, 227.6, 1, 121.4, 1572, 105.4, 891.8, 1285, 2187],
    'Language': ['English', 'English', 'English', 'English', 'English', 'English', 'French', 'German', 'Italian', 'Japanese'],
    'Currency': ['USD', 'USD', 'USD', 'USD', 'USD', 'GBP', 'EUR', 'EUR', 'EUR', 'JPY'],
    'Continent': ['North America', 'North America', 'North America', 'North America', 'North America', 'Europe', 'Europe', 'Europe', 'Europe', 'Asia'],
    'Is_Capital': [False, False, False, False, False, True, True, True, True, True]
}

cities_info = pd.DataFrame(data1)
cities_info

,City,Country,Population (Millions),Area (km2),Language,Currency,Continent,Is_Capital
0,New York,USA,8.4,468.9,English,USD,North America,False
1,Los Angeles,USA,3.9,502.8,English,USD,North America,False
2,Chicago,USA,2.7,227.6,English,USD,North America,False
3,Houston,USA,2.3,1.0,English,USD,North America,False
4,San Francisco,USA,0.9,121.4,English,USD,North America,False
5,London,UK,8.9,1572.0,English,GBP,Europe,True
6,Paris,France,2.1,105.4,French,EUR,Europe,True
7,Berlin,Germany,3.7,891.8,German,EUR,Europe,True
8,Rome,Italy,2.8,1285.0,Italian,EUR,Europe,True
9,Tokyo,Japan,13.9,2187.0,Japanese,JPY,Asia,True


In [6]:
data2 = {
    'City': ['New York', 'Chicago', 'Houston', 'London', 'Paris', 'Berlin', 'Rome', 'Tokyo', 'Lagos'],
    'Temperature (Celsius)': [18, 21, 30, 15, 19, 20, 24, 29, 34],
    'Humidity (%)': [65, 70, 75, 75, 80, 68, 72, 74, 83],
    'Rainfall (mm)': [50, 75, 20, 60, 40, 45, 35, 10, 122],
}

cities_weather = pd.DataFrame(data2)
cities_weather

,City,Temperature (Celsius),Humidity (%),Rainfall (mm)
0,New York,18,65,50
1,Chicago,21,70,75
2,Houston,30,75,20
3,London,15,75,60
4,Paris,19,80,40
5,Berlin,20,68,45
6,Rome,24,72,35
7,Tokyo,29,74,10
8,Lagos,34,83,122


### Challenge 1
Calculate the average population of cities in each continent using the groupby method

In [13]:
# Group by 'Continent' and calculate the mean of 'Population (Millions)'
average_population = cities_info.groupby('Continent')['Population (Millions)'].mean().reset_index()

average_population

,Continent,Population (Millions)
0,Asia,13.900
1,Europe,4.375
2,North America,3.640


### Challenge 2
Find the city with the highest population in each country using the groupby method and agg function

In [17]:
highest_population = cities_info.groupby('Country').agg('Population (Millions)').max().reset_index()
highest_population

,Country,Population (Millions)
0,France,2.1
1,Germany,3.7
2,Italy,2.8
3,Japan,13.9
4,UK,8.9
5,USA,8.4


### Challenge 3
Group the cities by their 'Language' and find the total area and total rainfall for each language group

In [20]:
# Merge the two DataFrames on 'City'
combined_data = pd.merge(cities_info, cities_weather, on='City', how='left')

# Group by 'Language' and calculate the total area and total rainfall
language_grouped = combined_data.groupby('Language').agg({
    'Area (km2)': 'sum',
    'Rainfall (mm)': 'sum'
}).reset_index()

language_grouped

,Language,Area (km2),Rainfall (mm)
0,English,2893.7,205.0
1,French,105.4,40.0
2,German,891.8,45.0
3,Italian,1285.0,35.0
4,Japanese,2187.0,10.0


### Challenge 4
Merge the cities_info and cities_weather DataFrames using a right join to keep only the cities for which weather data is available.

In [21]:
# Perform a right join on the 'City' column
merged_data = pd.merge(cities_info, cities_weather, on='City', how='right')

merged_data

,City,Country,Population (Millions),Area (km2),Language,Currency,Continent,Is_Capital,Temperature (Celsius),Humidity (%),Rainfall (mm)
0,New York,USA,8.4,468.9,English,USD,North America,False,18,65,50
1,Chicago,USA,2.7,227.6,English,USD,North America,False,21,70,75
2,Houston,USA,2.3,1.0,English,USD,North America,False,30,75,20
3,London,UK,8.9,1572.0,English,GBP,Europe,True,15,75,60
4,Paris,France,2.1,105.4,French,EUR,Europe,True,19,80,40
5,Berlin,Germany,3.7,891.8,German,EUR,Europe,True,20,68,45
6,Rome,Italy,2.8,1285.0,Italian,EUR,Europe,True,24,72,35
7,Tokyo,Japan,13.9,2187.0,Japanese,JPY,Asia,True,29,74,10
8,Lagos,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34,83,122


### Challenge 5
Calculate the sum of rainfall in each country using the groupby method.

In [22]:
# Merge the two DataFrames on 'City' using a right join
merged_data = pd.merge(cities_info, cities_weather, on='City', how='right')

# Group by 'Country' and calculate the sum of 'Rainfall (mm)'
rainfall_sum_by_country = merged_data.groupby('Country')['Rainfall (mm)'].sum().reset_index()

rainfall_sum_by_country

,Country,Rainfall (mm)
0,France,40
1,Germany,45
2,Italy,35
3,Japan,10
4,UK,60
5,USA,145


### Challenge 6
Group the cities by their 'Continent' and find the average temperature and humidity for each continent

In [23]:
# Group by 'Continent' and calculate the average temperature and humidity
average_temp_humidity_by_continent = merged_data.groupby('Continent').agg({
    'Temperature (Celsius)': 'mean',
    'Humidity (%)': 'mean'
}).reset_index()

average_temp_humidity_by_continent


,Continent,Temperature (Celsius),Humidity (%)
0,Asia,29.0,74.00
1,Europe,19.5,73.75
2,North America,23.0,70.00


### Challenge 7
Group the cities by their 'Is_Capital' and 'Language' columns and find the maximum population and area for each combination.

In [24]:
# Your code here# Group by 'Is_Capital' and 'Language' and find maximum population and area
max_pop_area_by_capital_language = cities_info.groupby(['Is_Capital', 'Language']).agg({
    'Population (Millions)': 'max',
    'Area (km2)': 'max'
}).reset_index()

max_pop_area_by_capital_language

,Is_Capital,Language,Population (Millions),Area (km2)
0,False,English,8.4,502.8
1,True,English,8.9,1572.0
2,True,French,2.1,105.4
3,True,German,3.7,891.8
4,True,Italian,2.8,1285.0
5,True,Japanese,13.9,2187.0
